In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Activation, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [5]:
pic_size = 48
batch_size = 128
base_path = "/content/content/opencv4/1"

# Aumento de datos para el conjunto de entrenamiento
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    os.path.join(base_path),
    target_size=(pic_size, pic_size),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
    os.path.join(base_path),
    target_size=(pic_size, pic_size),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

import os
print(os.listdir(base_path))

Found 0 images belonging to 1 classes.
Found 0 images belonging to 1 classes.
['Q.jpg', 'D.jpg', 'T.jpg', 'F.jpg', 'J.jpg', 'G.jpg', 'O.jpg', 'R.jpg', 'Y.jpg', 'Z.jpg', 'I.jpg', 'V.jpg', 'K.jpg', 'X.jpg', 'N.jpg', 'A.jpg', 'L.jpg', 'H.jpg', 'B.jpg', 'C.jpg', 'E.jpg', 'U.jpg', 'P.jpg', 'W.jpg', '.ipynb_checkpoints', 'M.jpg', 'S.jpg']


In [ ]:
nb_classes = 26

# Inicializando la CNN
model = Sequential()

# | 1. Capa de Convolución
model.add(Conv2D(64,(3,3), padding='same', input_shape=(48, 48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# | 2. Capa de Convolución
model.add(Conv2D(128,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# | 3. Capa de Convolución
model.add(Conv2D(256,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# | 4. Capa de Convolución
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Flatten())

# | 1. Capa Totalmente Conectada
model.add(Dense(128, kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))

# | 2. Capa Totalmente Conectada
model.add(Dense(256, kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))

# | 3. Capa Totalmente Conectada
model.add(Dense(512, kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))

# | 4. Capa Totalmente Conectada
model.add(Dense(256, kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))

# | 5. Capa Totalmente Conectada
model.add(Dense(128, kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(nb_classes, activation='softmax'))

# Inicialización del optimizador RMSprop
opt = RMSprop(lr=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Callbacks
checkpoint = ModelCheckpoint("model_4_weights.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=0.00001)
callbacks_list = [checkpoint, early_stopping, reduce_lr]

# Entrenamiento del modelo
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n//train_generator.batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.n//validation_generator.batch_size,
    callbacks=callbacks_list)

# Guardar el modelo y la arquitectura
model.save("model_4.h5")

json_config = model.to_json()
with open('arch_model_4.json', 'w') as json_file:
    json_file.write(json_config)

In [ ]:
plt.style.use("ggplot")
plt.figure(figsize=(20,10))
plt.subplot(1, 2, 1)
plt.ylabel('Loss', fontsize=16)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Cargar el modelo
model_4 = load_model("model_4.h5")

# Cargar y preprocesar la imagen
img = load_img('/content/disgust.jpg', target_size=(48, 48), color_mode="grayscale")
img = img_to_array(img)
img = img / 255.0
img = np.expand_dims(img, axis=0)

emotion_dict = {0: 'Felicidad', 1: 'Tristeza', 2: 'Sorpresa', 3: 'Enojo', 4: 'Disgusto', 5: 'Miedo', 6: 'Neutral'}

# Hacer la predicción
predictions = model_4.predict(img)

# Obtener la clase con la mayor probabilidad
class_index = np.argmax(predictions, axis=1)

# Obtener el nombre de la emoción
predicted_emotion = emotion_dict[class_index[0]]

print("La emoción predicha es:", predicted_emotion)